In [ ]:
import subprocess, os, contextlib
with contextlib.suppress(FileExistsError):
    os.mkdir('data')
# subprocess.check_call("gsutil -m cp gs://research-data-1432/180321-lp-gen-results/*.json data/", shell=True)

In [ ]:
from functools import partial
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import seaborn as sns
sns.set()
sns.set_style('white')

rename_columns = {
    'var_degree_max': 'Variable Degree Max',
    'cons_degree_max': 'Constraint Degree Max',
    'var_degree_min': 'Variable Degree Min',
    'cons_degree_min': 'Constraint Degree Min',
    'coefficient_density': 'Coefficient Density',
    'total_fractionality': 'Total Fractionality',
    'binding_constraints': 'Number of Binding Constraints',
    'rhs_mean': 'Constraint RHS Values Mean',
    'obj_mean': 'Objective Coefficients Mean',
    'clp_dual_iterations': 'Dual Simplex Iterations',
    'clp_barrier_iterations': 'Barrier Method Iterations',
    'clp_primal_iterations': 'Primal Simplex Iterations'
}

df_naive_random = pd.read_json('data/naive_random.json').rename(columns=rename_columns)
df_naive_random['Generator'] = 'Naive'
df_param_random = pd.read_json('data/parameterised_random.json').rename(columns=rename_columns)
df_param_random['Generator'] = 'Controlled'
df_naive_search = pd.read_json('data/naive_search.json').rename(columns=rename_columns)
df_param_search = pd.read_json('data/parameterised_search.json').rename(columns=rename_columns)
df_param_random.columns

with contextlib.suppress(FileExistsError):
    os.mkdir('figures')

def save(name):
    plt.savefig(f'figures/{name}.pdf')

palette = sns.color_palette()

## Naive generator parameters

3000 instances generated, each with a new random seed and the following parameters.
Each 'sample' generates parameters from these distributions and runs the naive algorithm with those parameters.

```py
size_params = dict(variables=50, constraints=50)
rhs_params = dict(
    rhs_mean=uniform(low=-100.0, high=100.0),
    rhs_std=uniform(low=1.0, high=30.0))
objective_params = dict(
    obj_mean=uniform(low=-100.0, high=100.0),
    obj_std=uniform(low=1.0, high=10.0))
lhs_params = dict(
    density=uniform(low=0.0, high=1.0),
    pv=uniform(low=0.0, high=1.0),
    pc=uniform(low=0.0, high=1.0),
    coeff_loc=uniform(low=-2.0, high=2.0),
    coeff_scale=uniform(low=0.1, high=1.0))
```

## Controlled generator parameters

1000 instances generated, as above, using the controlled algorithm.
Generated more naive instances to ensure we had at least 1000 feasible, bounded instances as a comparison set.

```py
size_params = dict(variables=50, constraints=50)
beta_params = dict(
    basis_split=random_state.uniform(low=0.0, high=1.0))
alpha_params = dict(
    frac_violations=random_state.uniform(low=0.0, high=1.0),
    beta_param=random_state.lognormal(mean=-0.2, sigma=1.8),
    mean_primal=0,
    std_primal=1,
    mean_dual=0,
    std_dual=1)
lhs_params = dict(
    density=random_state.uniform(low=0.0, high=1.0),
    pv=random_state.uniform(low=0.0, high=1.0),
    pc=random_state.uniform(low=0.0, high=1.0),
    coeff_loc=random_state.uniform(low=-2.0, high=2.0),
    coeff_scale=random_state.uniform(low=0.1, high=1.0))
```

In [ ]:
print(f'Controlled generator samples: {df_param_random.shape[0]}')
print(f'Naive generator samples: {df_naive_random.shape[0]}')
print(f'Naive generator feasible bounded samples: {df_naive_random.solvable.sum()}')
print(f'Probability naive feasible and bounded: {df_naive_random.solvable.mean():.3f}')

## Generator Results

\Cref{fig:feature_dist} shows the variance in feature values achieved by the generators for feasible, bounded instances.
This shows generated feasible, bounded instances only.
Separate data sets are only shown where the feature distributions differ (specifically, the same generation algorithm is used for the constraint matrix in both cases, therefore all relevant features are sampled from identical distributions).

\Cref{fig:feature_dist:density,fig:feature_dist:degree} show variation in features of the constraint matrix.
By constructing (approximate) degree sequences for the variable-constraint graph the degree statistics can be varied ~independently of the density and each other.

In [ ]:
sns.lmplot(
    data=df_param_random,
    x='Coefficient Density',
    y='Variable Degree Max',
    fit_reg=False)
save('feature-dist-density')

In [ ]:
ax = sns.lmplot(
    data=df_param_random,
    x='Constraint Degree Max',
    y='Variable Degree Max',
    fit_reg=False)
save('feature-dist-degree')

\Cref{fig:feature-dist:relaxation} shows the joint distribution of number of binding constraints and total fractionality of the solutions.
Here we see the main additional parameters of the controlled generator - we can set the structure of the primal-dual solution.
Refer to derivations in section 4.
By contrast, the naive generator occupies a very narrow band in this space.
More importantly, the parameters of this generator have no bearing on these features, so we cannot target a location.

In [ ]:
ax = sns.lmplot(
    data=pd.concat([
        df_param_random,
        df_naive_random[df_naive_random.solvable]]),
    x='Number of Binding Constraints',
    y='Total Fractionality',
    hue='Generator',
    hue_order=['Controlled', 'Naive'],
    palette=reversed(palette[0:2]),
    fit_reg=False)
ax.ax.set_ybound(lower=-1, upper=26)
save('feature-dist-relaxation')

\Cref{fig:feature-dist:obj-rhs} shows the joint distribution of mean obj/rhs.
Note that the parameters of the naive generator distribute this feature jointly uniformly, but we have omitted any infeasible or unbounded instances produced by this generator.
With the parameter distribution we have chosen for the controlled generator, there is a strong correlation between the two features shown here.
While the naive generator used uncorrelated data, there is a clear relationship between location in this feature space and feasibility.
Note in our choice of parameters, there is less variance in A than in b or c.
We limit that range to ensure we are not simply generating scaled problems...?

As previously mentioned, 37% of generated instances using the naive generator were feasible.
This particular feature space seems ideal to observe the transition.

In [ ]:
ax = sns.lmplot(
    data=pd.concat([
        df_param_random,
        df_naive_random[df_naive_random.solvable]]),
    x='Constraint RHS Values Mean',
    y='Objective Coefficients Mean',
    hue='Generator',
    hue_order=['Naive', 'Controlled'],
    palette=palette[0:2],
    fit_reg=False)
save('feature-dist-obj-rhs')

\Cref{fig:naive-transition:scatter} shows the sample points of the naive generator.
\Cref{fig:naive-transition:prob} shows the probability that an instance generated by the naive generator, and observed to be at a given point in the feature space, is feasible and bounded.
This shows the transitional regions.
This estimation of probability is specific to the parameter distribution we have chosen here.


* Figures below show the phase transition observed in instances produces by the naive generator.
* Since the naive method generates instances by varying b and c, we vary these properties uniformly.
* Second figure shows the probability that an instance is feasible and bounded given its location (this is an observation very specific to this distribution...)

We should note that:
* stdev is comparatively small compared to the range in the means, and is constant. This is chosen in order to observe this phase transition. Scaling both together would not yield anything of interest - at base level we need to vary the likelihood of the obj/rhs coefficients being positive or negative.
* likewise the values in A are being varied independently.

A couple of obvious observations:
* if all b > 0, primal is zero-feasible. if all c < 0, dual is zero-feasible. In this case the instance is clearly feasible and bounded. This quadrant is likely to be uninteresting - x = 0 is the likely solution.

In [ ]:
df_naive_random['Feasible and Bounded'] = df_naive_random['solvable'].apply(
    lambda v: 'Yes' if v else 'No')
sns.lmplot(
    data=df_naive_random,
    x='Constraint RHS Values Mean',
    y='Objective Coefficients Mean',
    hue='Feasible and Bounded',
    hue_order=['Yes', 'No'],
    palette=palette[2:],
    size=5, fit_reg=False, scatter_kws={'alpha': 0.6})
save('naive-transition-scatter')

In [ ]:
df = df_naive_random[df_naive_random.solvable]
sns.jointplot(
    x=df['Constraint RHS Values Mean'],
    y=df['Objective Coefficients Mean'],
    kind='kde', color='k', size=5, stat_func=None)
save('naive-transition-prob')

Observing the difficulty of the naive instances - we see that that 'boring' region corresponds to easily solve instances. The transitional regions, where feasibility is less predictable, tends to produce harder FB instances.
Furthermore, the parameterised generator, which produces instances mostly in that region, produces harder instances.
This may be also partially explained by observing that the parameterised generator controls number of binding constranits, which seems to influence difficulty directly.

\Cref{fig:hardness:naive} shows the difficulty of generated instances, by measuring number of dual simplex iterations required to solve each instance.
Observe in the 'predictable' region, instances are extremely easy, in fact they are often solved by presolve.
Harder instances occur in the transitional regions.

In [ ]:
df = df_naive_random[df_naive_random.solvable].sample(1000)
print(f'Naive Generator. F+B samples: {df.shape[0]}')
fig, ax = plt.subplots()
sc = ax.scatter(
    x=df['Constraint RHS Values Mean'],
    y=df['Objective Coefficients Mean'],
    c=df['Dual Simplex Iterations'], cmap=cm.coolwarm)
ax.set_xbound([-125, 125])
ax.set_ybound([-125, 125])
ax.set_xlabel('Constraint RHS Values Mean')
ax.set_ylabel('Objective Coefficients Mean')
cb = plt.colorbar(sc)
cb.set_label('Dual Simplex Iterations')
sns.despine()
save('hardness-naive')

\Cref{fig:hardness:controlled} shows the difficult of instances produced with the controlled generator.
Since this produces instances mainly in the transitional region, instances tend to be harder.

In [ ]:
df = df_param_random[df_param_random.solvable]
print(f'Parameterised Generator. Samples: {df.shape[0]}')
fig, ax = plt.subplots()
sc = ax.scatter(
    x=df['Constraint RHS Values Mean'],
    y=df['Objective Coefficients Mean'],
    c=df['Dual Simplex Iterations'], cmap=cm.coolwarm)
ax.set_xbound([-125, 125])
ax.set_ybound([-125, 125])
ax.set_xlabel('Constraint RHS Values Mean')
ax.set_ylabel('Objective Coefficients Mean')
cb = plt.colorbar(sc)
cb.set_label('Dual Simplex Iterations')
sns.despine()
save('hardness-controlled')

Indeed if we observe the relative distributions in \Cref{generated-hardness-hist} we see that the controllable generator produces less junk and also has a harder top-end.

In [ ]:
key = 'Dual Simplex Iterations'
x = [
    list(df_naive_random.loc[df_naive_random.solvable, key]),
    list(df_param_random.loc[df_param_random.solvable, key]),
]
plt.hist(x, bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

In [ ]:
(df_naive_random['Dual Simplex Iterations'] < 10).sum()

In [ ]:
def bin_it(data):
    key = 'Dual Simplex Iterations'
    bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    df = data.loc[data.solvable, key]
    grouped = df.groupby(pd.cut(df, bins=bins)).count()
    grouped.index = bins[:-1]
    return grouped

df = pd.DataFrame({
    'Naive': bin_it(df_naive_random),
    'Controlled': bin_it(df_param_random)})
plt.bar(x=list(df.index), height=df['Naive'], width=5)
plt.bar(x=list(df.index + 5), height=df['Controlled'], width=5)
df

In [ ]:
hist = partial(plt.hist, density=True, alpha=0.8)

df1 = df_naive_random[df_naive_random.solvable]
df2 = df_param_random[df_param_random.solvable]
hist(df1['Dual Simplex Iterations'], label='Naive')
hist(df2['Dual Simplex Iterations'], label='Controlled')
plt.xlabel('Dual Simplex Iterations')
plt.ylabel('Frequency')
plt.legend()
sns.despine()
save('generated-hardness-hist')

Further observation of \Cref{fig:binding-constraints-hardness}, which shows difficulty for dual simplex against number of binding constraints, indicates part of the reason for this.
The controllable generator produces instances with more binding constraints, and this is strongly correlated with difficulty.

In [ ]:
df1 = df_naive_random[df_naive_random.solvable].sample(1000)
df2 = df_param_random[df_param_random.solvable]
df1['Generator'] = 'Naive'
df2['Generator'] = 'Controlled'
sns.lmplot(
    data=pd.concat([df1, df2]),
    x='Number of Binding Constraints',
    y='Dual Simplex Iterations',
    hue='Generator',
    hue_order=['Controlled', 'Naive'],
    palette=reversed(palette[0:2]),
    fit_reg=False,
    scatter_kws={'alpha': 0.6})
save('binding-constraints-hardness')

Examination of figures ... shows:
* comparatively, the controllable generator produces more feature variation,
* harder instances,
* varies features correlated with difficulty (solution structure).

From the point of view of generating a test set we observe two issues with the results so far:
* We are missing some regions of feature space (we do not have examples there) for feasible bounded instances.
To test a solver effectively we want to ensure we have varied all parameters of interest, so we should use search to explore.
* It is likely we can find more variation in performance space.

The next section uses search for both...

# Search

## Searching feature space

In all runs, the objective is simply to reach the target point (-100, 100) in the obj/rhs feature space (our missing region).
A start point is selected by randomly sampling 10 instances from the existing FB set and choosing the closest point of those to the target.
At each local search step:
* generates a neighbour (using either naive or controllable method)
* accepts that neighbour if it is an improvement.

If the naive operator produces an infeasible or unbounded instance, it is rejected.

Search continues for 10000 of these steps.

We repeat each search process 100 times.

\Cref{feature-search-target} shows the original data set, and sampled points for each search run at step 1000 (mainly to show the target point).

## Searching performance space

Should do a best of 1000 for the start point - we are not choosing a difficult enough start.

Rundown of search - the figure below shows state of play after 1000 steps.
Subsequent figure shows reduced distance to target after N steps, clearly showing param search is more effective.
Histograms show that we don't really find harder instances, but we do eliminate the noise of easy instances by looking in this area where we don't expect to find this instance class.

In [ ]:
df1 = df_naive_random[df_naive_random.solvable].sample(1000).copy()
df1['Source'] = 'Naive Generator'
df2 = df_param_random[df_param_random.solvable].copy()
df2['Source'] = 'Controlled Generator'

step = 1000
df3 = df_naive_search[df_naive_search.solvable & (df_naive_search.step == step)].copy()
df3['Source'] = 'Naive Search'
df4 = df_param_search[df_param_search.solvable & (df_param_search.step == step)].copy()
df4['Source'] = 'Controlled Search'

g = sns.lmplot(
    data=pd.concat([df1, df2, df3, df4]),
    x='Constraint RHS Values Mean',
    y='Objective Coefficients Mean',
    hue='Source',
    fit_reg=False,
    hue_order=[
        'Naive Generator', 'Controlled Generator',
        'Naive Search', 'Controlled Search'],
    palette=palette
)
g.ax.set_xbound([-125, 125])
g.ax.set_ybound([-125, 125])
save('feature-search-target')

\Cref{feature-search-progression} compares performance of naive and controlled search.
Naive search has ~50% rejection rate due to producing inf/ub instances.
Controlled search is hence more efficient for searching P.

In [ ]:
def calc_objective(df):
    objective = (
        (df['Constraint RHS Values Mean'] + 100) ** 2 +
        (df['Objective Coefficients Mean'] - 100) ** 2) ** 0.5
    return pd.DataFrame(dict(step=df.step, objective=objective))

def compare_trajectory(labelled_series, ax, palette):
    ''' Expects series (step, objective) for comparison. '''
    assert len(labelled_series.items()) == 2
    for (label, data), color, linestyle in zip(labelled_series.items(), palette, ['--', '-']):
        df = data.groupby('step').objective.apply(
            lambda x: {
                'median': x.median(),
                'lower': x.quantile(0.25),
                'upper': x.quantile(0.75)}
            ).unstack(1).reset_index()
        ax.fill_between(df['step'], df['lower'], df['upper'], alpha=0.3, color=color)
        ax.plot(df['step'], df['median'], linestyle, color=color, label=label)
    ax.collections[0].set_hatch('xx')
    ax.set_xlabel('Search Step')
    ax.legend()
    return ax

In [ ]:
fig, ax = plt.subplots()
ax = compare_trajectory({
    'Naive': calc_objective(df_naive_search),
    'Controlled': calc_objective(df_param_search)
    }, ax, palette)
ax.semilogy()
ax.set_ylabel('Distance to Target')
sns.despine(fig)
save('feature-search-progression')

\Cref{fig:feature-search-hardness} shows that the search processes generally produced non-trivial instances but did not find anything harder than what we could already generate.
Therefore, we should also apply search to attempt to improve the top end.

In [ ]:
df1 = df_naive_random[df_naive_random.solvable].sample(1000).copy()
df2 = df_param_random[df_param_random.solvable].copy()

step = 10000
df3 = df_naive_search[df_naive_search.solvable & (df_naive_search.step == step)].copy()
df4 = df_param_search[df_param_search.solvable & (df_param_search.step == step)].copy()

hist = partial(plt.hist, density=True, alpha=0.8)
hist(df1['Dual Simplex Iterations'], label='Naive Generator')
hist(df2['Dual Simplex Iterations'], label='Parameterised Generator')
hist(df3['Dual Simplex Iterations'], label='Naive Search')
hist(df4['Dual Simplex Iterations'], label='Parameterised Search')
plt.xlabel('Dual Simplex Iterations')
plt.ylabel('Frequency')
plt.legend()
sns.despine()
save('feature-search-hardness')

In [ ]:
df1 = df_naive_random[df_naive_random.solvable].sample(1000).copy()
df2 = df_param_random[df_param_random.solvable].copy()

step = 10000
df3 = df_naive_search[df_naive_search.solvable & (df_naive_search.step == step)].copy()
df4 = df_param_search[df_param_search.solvable & (df_param_search.step == step)].copy()

key = 'Dual Simplex Iterations'
sns.boxplot(data=pd.DataFrame({
    'Naive\nGenerator': df1[key].reset_index(drop=True),
    'Controlled\nGenerator': df2[key].reset_index(drop=True),
    'Naive\nSearch': df3[key].reset_index(drop=True),
    'Controlled\nSearch': df4[key].reset_index(drop=True),
}), orient='h')
plt.xlabel('Dual Simplex Iterations')
plt.tight_layout()
save('feature-search-hardness-dual-boxplot')

In [ ]:
key = 'Primal Simplex Iterations'
sns.boxplot(data=pd.DataFrame({
    'Naive\nGenerator': df1[key].reset_index(drop=True),
    'Controlled\nGenerator': df2[key].reset_index(drop=True),
    'Naive\nSearch': df3[key].reset_index(drop=True),
    'Controlled\nSearch': df4[key].reset_index(drop=True),
}), orient='h')
plt.xlabel('Primal Simplex Iterations')
plt.tight_layout()
save('feature-search-hardness-primal-boxplot')

In [ ]:
def plot_for_field(ax, field):
    datasets = {
        'Naive': pd.read_json(
            f'data/naive_performance_search_{field}.json')[['step', field]].rename(
            columns={field: 'objective'}),
        'Controlled': pd.read_json(
            f'data/parameterised_performance_search_{field}.json')[['step', field]].rename(
            columns={field: 'objective'})}
    ax = compare_trajectory(datasets, ax, palette)
    return ax

In [ ]:
fig, ax = plt.subplots()
ax = plot_for_field(ax, 'clp_primal_iterations')
ax.set_ylabel('Primal Simplex Iterations')
sns.despine(fig)
plt.legend(loc=4)
save('search-primal')

In [ ]:
fig, ax = plt.subplots()
ax = plot_for_field(ax, 'clp_dual_iterations')
ax.set_ylabel('Dual Simplex Iterations')
sns.despine(fig)
save('search-dual')
plt.legend(loc=4)

In [ ]:
fig, ax = plt.subplots()
ax = plot_for_field(ax, 'clp_barrier_iterations')
ax.set_ylabel('Barrier Method Iterations')
sns.despine(fig)
save('search-barrier')
plt.legend(loc=4)

In [ ]:
cols = ['Primal Simplex Iterations', 'Dual Simplex Iterations', 'Barrier Method Iterations']

df1 = df_naive_random.loc[df_naive_random.solvable, cols]
df2 = df_param_random.loc[df_param_random.solvable, cols]

result1 = pd.DataFrame({
    ('Naive Generator', None, 'Q1'): df1.quantile(.25),
    ('Naive Generator', None, 'Q2'): df1.median(),
    ('Naive Generator', None, 'Q3'): df1.quantile(.75),
    ('Controllable Generator', None, 'Q1'): df2.quantile(.25),
    ('Controllable Generator', None, 'Q2'): df2.median(),
    ('Controllable Generator', None, 'Q3'): df2.quantile(.75),
}).transpose().unstack()
result1

In [ ]:
def _read(method, field):
    full_df = pd.read_json(f'data/{method}_performance_search_{field}.json')
    for step in [0, 200, 500, 1000]:
        df = full_df.loc[full_df.step == step, field]
        yield dict(
            Q1=df.quantile(.25), Q2=df.median(), Q3=df.quantile(.75),
            step=step, field=field, method=method)

import itertools

result = pd.DataFrame(list(itertools.chain(*(
    _read(method, field)
    for method, field in itertools.product(
        ['naive', 'parameterised'],
        ['clp_primal_iterations', 'clp_dual_iterations', 'clp_barrier_iterations']
    )))))
result['field'] = result['field'].map({
    'clp_dual_iterations': 'Dual Simplex Iterations',
    'clp_barrier_iterations': 'Barrier Method Iterations',
    'clp_primal_iterations': 'Primal Simplex Iterations'
})
result['method'] = result['method'].map({
    'naive': 'Naive Search',
    'parameterised': 'Controllable Search'
})
result = result.set_index(['field', 'method', 'step']).stack().unstack(0).unstack(2)
result = result[[
    'Primal Simplex Iterations',
    'Dual Simplex Iterations',
    'Barrier Method Iterations',
    ]]
result = pd.concat([result, result1]).sort_index()
result = (
    result
    .rename(columns=lambda col: col.replace(' Iterations', ''))
    .astype('int')
)
print(result.to_latex())
result

In [ ]:
# TODO add random dist values to each of these.

steps = [0, 100, 1000, 10000]

def _read():
    full_df = calc_objective(df_param_search)
    for step in steps:
        df = full_df.loc[full_df.step == step, 'objective']
        yield dict(method='Controllable Search', min=df.min(), median=df.median(), step=step)
    full_df = calc_objective(df_naive_search)
    for step in steps:
        df = full_df.loc[full_df.step == step, 'objective']
        yield dict(method='Naive Search', min=df.min(), median=df.median(), step=step)

result = pd.DataFrame(list(_read())).set_index(['method', 'step'])
print(result.round(3).to_latex())